In [74]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,OneHotEncoder, StandardScaler
import pickle


In [75]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [76]:
## Preprocessing the data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [77]:
#Encode the categorical variable
Label_encoder_gender=LabelEncoder()
data['Gender']=Label_encoder_gender.fit_transform(data['Gender'])


In [78]:
#Encode the One hot encoder for places values
one_hot_encoder=OneHotEncoder()
one_hot_encoder_geo=one_hot_encoder.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(one_hot_encoder_geo,columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [79]:
#Combine the data frame
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [80]:
## Split the data into independent features and target features
X=data.drop('EstimatedSalary',axis=1)
Y=data['EstimatedSalary']


In [81]:
# Split into train test split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
assert (X_train.columns == X_test.columns).all(), "Column names or order mismatch"

# Standard scaler applied to test train split
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [82]:
# Save the encoder and scaler for future use

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(Label_encoder_gender,file)

with open('one_hot_encoder_geo','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)


with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)
    


## ANN Regression problem statement


In [84]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [85]:
## Build the model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)  ## output layer for regression

          
])

## compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])


model.summary()

e:\GenAI LLM 2024\venv2\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [86]:
## Using the tensorboard for logs
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
import datetime
log_dir="regressionlogs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callbacks=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [87]:
## Early stopping based on patience level
earlystopping_callbacks=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [88]:
## train the model
history=model.fit(
    X_train,Y_train,validation_data=(X_test,Y_test),
    epochs=100,
    callbacks=[tensorboard_callbacks,earlystopping_callbacks]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 100888.4922 - mae: 100888.4922 - val_loss: 98544.5703 - val_mae: 98544.5703
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 100000.4844 - mae: 100000.4844 - val_loss: 97175.4297 - val_mae: 97175.4297
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 98747.8516 - mae: 98747.8516 - val_loss: 93638.9219 - val_mae: 93638.9219
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 92889.1719 - mae: 92889.1719 - val_loss: 87655.4688 - val_mae: 87655.4688
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 87947.3438 - mae: 87947.3438 - val_loss: 79826.0234 - val_mae: 79826.0234
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 79652.8906 - mae: 79652.8906 - val_loss: 71475.1328 - val_mae: 71475.1328
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 69993.7344 - mae: 69993.7344 - val_loss: 63836.7422 - val_mae: 63836.7422
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/ste

In [89]:
## models save

model.save('regression_model.h5')

In [59]:
one_hot_encoder_geo.Categories_[0]

AttributeError: 'numpy.ndarray' object has no attribute 'Categories_'

In [64]:
Y

0       101348.88
1       112542.58
2       113931.57
3        93826.63
4        79084.10
          ...    
9995     96270.64
9996    101699.77
9997     42085.58
9998     92888.52
9999     38190.78
Name: EstimatedSalary, Length: 10000, dtype: float64